# Activity 3

In [ ]:
# Modified query to find sediments (sampling point and other information not required)
q = '''
SELECT ?name ?sedimentname
WHERE {

    SERVICE <http://data.ordnancesurvey.co.uk/datasets/boundary-line/apis/sparql> { 
        ?area rdfs:label "North Yorkshire" .
        ?district ossr:within ?area .
        ?district rdfs:label ?districtname .
    }
    
    ?location a <http://environment.data.gov.uk/def/bathing-water/BathingWater> .
    ?location <http://environment.data.gov.uk/def/bathing-water/sedimentTypesPresent> ?sediment .
    ?location <http://statistics.data.gov.uk/def/administrative-geography/district> ?district .
    ?location rdfs:label ?name .
    ?sediment rdfs:label ?sedimentname .
         
    FILTER(LANG(?sedimentname) = "" || LANGMATCHES(LANG(?sedimentname), "en"))
    
}
'''

# Run query
df = dfResults(endpoint_envAgency, prefix, q)

# Set up series for all sediment types to count occurrences
s = pd.Series([0,0,0,0,0,0], index=['sand', 'shingle', 'rock', 'marsh', 'mud', 'other'])   

# Loop through the sediment 
for ix, row in df[['sedimentname']].iterrows(): 
    if (row['sedimentname'] == 'sand'):
        s['sand'] = s['sand']+1
    elif (row['sedimentname'] == 'shingle'):
        s['shingle'] = s['shingle'] + 1
    elif (row['sedimentname'] == 'rock'):
        s['rock'] = s['rock'] + 1
    elif (row['sedimentname'] == 'marsh'):
        s['marsh'] = s['marsh'] + 1
    elif (row['sedimentname'] == 'mud'):
        s['mud'] = s['mud'] + 1
    else:
        s['other'] = s['other']+1
        
print(s)

# Plot results
s.plot(kind='pie')

# Activity 4

In [ ]:
q1 = query('stoke-on-trent')
df1 = dfResults(endpoint_lr,prefix,q1)

q2 = query('york')
df2 =  dfResults(endpoint_lr,prefix,q2)

q3 = query('poole')
df3 =  dfResults(endpoint_lr,prefix,q3)

frames = [df1, df2, df3]
df = pd.concat(frames, ignore_index=True)

df = df.apply(pd.to_numeric)
df.index = ['Stoke-on-Trent', 'York', 'Poole']

print(df)

df.plot(kind='bar')

From this data it is clear that the average house prices in Stoke-on-Trent were considerably lower than those in the more affluent areas of York and Poole.

# Activity 5

Our attempt illustrates some futher features of the `plot()` function.

In [ ]:
endpoint_lr = 'http://landregistry.data.gov.uk/landregistry/query'

prefix = '''
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX lrhpi: <http://landregistry.data.gov.uk/def/hpi/>
'''

# Amend the the query to specify the local authority name and the required month
# VALUES statement requires amending
def query(name, month):
    s = '''
    SELECT DISTINCT  ?avgDetached
    WHERE{
    
        VALUES ?localAuthorityMonth {<http://landregistry.data.gov.uk/data/hpi/region/'''
    s = s + str(name) + "/month/2014-" + str(month) + '''>
    }

  ?localAuthorityMonth
    lrhpi:refRegion ?regionURI ;
    lrhpi:averagePricesDetachedSASM ?avgDetached .
    
    
  ?regionURI rdfs:label ?regionName .
  FILTER (langMatches(lang(?regionName), "EN") )
}
'''
    return s
    
# Loop through months; add result to series y
# The month is specified as a two character string, '01' to '12'
y = []
x = []
for m in range(1, 13):
    month = str(m)
    if len(month) == 1:
        month = '0' + month
    q2 = query('city-of-plymouth', month)   
    df2 = dfResults(endpoint_lr, prefix, q2)
    y.append(df2.iloc[0,0]) # House price
    x.append(m) # Month

# Find the maximum and minimum prices
miny = int(min(y))
maxy = int(max(y))

# Set up the axes for the graph (range of values and labels)
plt.axis([1, 12, miny, maxy])
plt.xlabel('Month')
plt.ylabel('Price')

# Give the graph a title
plt.title('Average Detached House Prices 2014 for Plymouth')

# Plot the graph using blue lines (instead of the default red)
plt.plot(x, y, 'b')

# Some more results
print('Yearly change: £', (int(y[11])- int(y[0])))
print('Minimum average price: £', miny)
print('Maximum average price: £', maxy)